Robot Localization

In [35]:
p = [0.2, 0.2, 0.2, 0.2, 0.2] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = "red" # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))           
    return belief
print(sense(p, Z))

[0.04000000000000001, 0.12, 0.12, 0.04000000000000001, 0.04000000000000001]


Normalizing so that the probailities add up to one

In [36]:
sumPosterior = sum(sense(p, Z))
print ("Sum of Posterior: " + str(sumPosterior))
normProb = []
for i in sense(p, Z):
    normProb.append(i/sumPosterior)
print("Measurement Update: " + str(normProb)) # measurement update
print("Sum of Normalized Probaility: " + str(sum(normProb)))

Sum of Posterior: 0.3600000000000001
Measurement Update: [0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]
Sum of Normalized Probaility: 0.9999999999999998


---------------------- OR -------------------------

In [37]:
p = [0.2, 0.2, 0.2, 0.2, 0.2] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = "red" # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief
print(sense(p, Z))

[0.1111111111111111, 0.3333333333333332, 0.3333333333333332, 0.1111111111111111, 0.1111111111111111]


What if the robot Senses twice ie red and green.

In [38]:
p = [0.2, 0.2, 0.2, 0.2, 0.2] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "green"] # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief
    
for k in range(len(Z)):
    p = sense(p, Z[k]) # recursion
print(p)

[0.20000000000000004, 0.19999999999999996, 0.19999999999999996, 0.20000000000000004, 0.20000000000000004]


### ROBOT MOTION

EXACT MOTION

In [39]:
p = [0, 1, 0, 0, 0] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "green"] # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief

def move(p, U):  # U is the number of times the robot moves to the right or left
    q = []
    for i in range(len(p)):
        q.append(p[i-U % len(p)])  #the modulus operator is used incase the shift exceed the len of p
    return q
# for k in range(len(Z)):
#     p = sense(p, Z[k]) # recursion
print(move(p, 2))

[0, 0, 0, 1, 0]


INEXACT MOTION

In [40]:
p = [0, 0.5, 0, 0.5, 0] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "green"] # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty
pExact = 0.8 # two steps
pOvershoot = 0.1 # three steps
pUndershoot = 0.1 # one step

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief

def move(p, U):  # U is the number of times the robot moves to the right or left
    q = []
    for i in range(len(p)):
        prob_of_movement = pExact * p[i-U % len(p)]
        prob_of_movement = prob_of_movement + pOvershoot * p[i-U-1 % len(p)] # not yet understood
        prob_of_movement = prob_of_movement + pUndershoot * p[i-U+1 % len(p)]
        q.append(prob_of_movement)
    return q

# for k in range(len(Z)):
#     p = sense(p, Z[k]) # recursion
print(move(p, 2))

[0.4, 0.05, 0.05, 0.4, 0.1]


Inexact motion with  robot that moves twice

In [41]:
p = [0, 1, 0, 0, 0] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "green"] # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty
pExact = 0.8 # two steps
pOvershoot = 0.1 # three steps
pUndershoot = 0.1 # one step
z = []

def sense(prob, measurement): # posterior
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief

def move(p, U):  # U is the number of times the robot moves to the right or left
    q = []
    for i in range(len(p)):
        prob_of_movement = pExact * p[i-U % len(p)]
        prob_of_movement = prob_of_movement + pOvershoot * p[i-U-1 % len(p)] # not yet understood
        prob_of_movement = prob_of_movement + pUndershoot * p[i-U+1 % len(p)]
        q.append(prob_of_movement)
        
    return q

# for k in range(len(Z)):
#     p = sense(p, Z[k]) # recursion

z = move(p, 1)
z = move(z, 1)
print(z)
# OR
# p = move(p, 1)
# p = move(p, 1)
# print(p)

[0.010000000000000002, 0.010000000000000002, 0.16000000000000003, 0.6600000000000001, 0.16000000000000003]


Now, Lets move 1000 times

In [42]:
p = [0, 1, 0, 0, 0] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "green"] # robot measurement
pHit = 0.6 # reward
pMiss = 0.2 # penalty
pExact = 0.8 # two steps
pOvershoot = 0.1 # three steps
pUndershoot = 0.1 # one step
z = []

# def sense(prob, measurement):
#     belief = [] # unbalanced probability
#     for i in range(len(prob)):
#         hit = (measurement == world[i])
#         belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
#     s = sum(belief)
#     for j in range(len(belief)):
#         belief[j] = belief[j]/s
#     return belief

def move(p, U):  # U is the number of times the robot moves to the right or left
    q = []
    for i in range(len(p)):
        prob_of_movement = pExact * p[i-U % len(p)]
        prob_of_movement = prob_of_movement + pOvershoot * p[i-U-1 % len(p)] # not yet understood
        prob_of_movement = prob_of_movement + pUndershoot * p[i-U+1 % len(p)]
        q.append(prob_of_movement)
        
    return q

# for k in range(len(Z)):
#     p = sense(p, Z[k]) # recursion

for i in range(1000):
    p = move(p, 1)
print(p)

[0.20000000000000365, 0.20000000000000373, 0.20000000000000365, 0.2000000000000035, 0.2000000000000035]


We notice that the result is a uniform distribution

### SENSE AND MOVE

Now, we want to robot to sense red and move right then sense green and move right again. Assume uniform distribution

In [48]:
p = [0.2, 0.2, 0.2, 0.2, 0.2] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "green"] # robot measurement
motion = [1, 1]
pHit = 0.6 # reward
pMiss = 0.2 # penalty
pExact = 0.8 # two steps
pOvershoot = 0.1 # three steps
pUndershoot = 0.1 # one step
z = []

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief

def move(p, U):  # U is the number of times the robot moves to the right or left
    q = []
    for i in range(len(p)):
        prob_of_movement = pExact * p[i-U % len(p)]
        prob_of_movement = prob_of_movement + pOvershoot * p[i-U-1 % len(p)] # not yet understood
        prob_of_movement = prob_of_movement + pUndershoot * p[i-U+1 % len(p)]
        q.append(prob_of_movement)
        
    return q

for k in range(len(Z)):
    p = sense(p, Z[k]) # recursion
    p = move(p, motion[k])
print(p)

[0.21157894736842103, 0.1515789473684211, 0.08105263157894739, 0.16842105263157897, 0.3873684210526316]


The robot after moving twice has a higher probablility of 0.39 that it is closer to a green

- What if the robot senses red twice?

In [49]:
p = [0.2, 0.2, 0.2, 0.2, 0.2] # uniform distribution (Prior)
world = ["green", "red", "red", "green", "green"]
Z = ["red", "red"] # robot measurement
motion = [1, 1]
pHit = 0.6 # reward
pMiss = 0.2 # penalty
pExact = 0.8 # two steps
pOvershoot = 0.1 # three steps
pUndershoot = 0.1 # one step
z = []

def sense(prob, measurement):
    belief = [] # unbalanced probability
    for i in range(len(prob)):
        hit = (measurement == world[i])
        belief.append(prob[i] * ((hit * pHit) + ((1-hit) * pMiss)))  
    s = sum(belief)
    for j in range(len(belief)):
        belief[j] = belief[j]/s
    return belief

def move(p, U):  # U is the number of times the robot moves to the right or left
    q = []
    for i in range(len(p)):
        prob_of_movement = pExact * p[i-U % len(p)]
        prob_of_movement = prob_of_movement + pOvershoot * p[i-U-1 % len(p)] # not yet understood
        prob_of_movement = prob_of_movement + pUndershoot * p[i-U+1 % len(p)]
        q.append(prob_of_movement)
        
    return q

for k in range(len(Z)):
    p = sense(p, Z[k]) # recursion
    p = move(p, motion[k])
print(p)

[0.07882352941176471, 0.07529411764705884, 0.22470588235294123, 0.4329411764705882, 0.18823529411764706]


The robot after sensing red twice, moved a step to the right (i.e. the fourth cell) with a higher probability of 0.43

This is similar to the system used in Google's self-driving cars. 
But instead of the color red and green, we would plug in the color of the lane markings relative to the color of the pavement

SO IN SUMMARY
- BELIEF = PROBABLILTY
- SENSE = PRODUCT followed by NORMALIZATION
- MOVE = CONVOLUTION (ADDITION)